# Job Search

How do we create a job search using redis?

- jobs can be stored in sets
- each job will have the skills required as the member
- candidate skills can be added to sets
- the difference between the job and candidate sets will determine if the candidate is a match
- if the difference is zero, it means the candidate has all the skills required
- this requires us to check the candidate against all jobs, which might not scale

In [3]:
def add_job(conn, job_id, required_skills):
    conn.sadd(f'job:{job_id}', *required_skills)

In [ ]:
def is_qualified(conn, job_id, candidate_skills):
    temp = str(uuid.uuid4())
    pipeline = conn.pipeline(True)
    pipeline.sadd(temp, *candidate_skills)
    pipeline.expire(temp, 5)
    pipeline.sdiff(f'job:{job_id}', temp)
    return not pipeline.execute()[-1]

## Approaching the problem like search

- we can use inverted index to improve the performance
- we create a SET for each skill, and the members are the jobs id
- we use a ZSET to store the required skills for each job

How does the search works?

- we have a SET for the candidate's skills
- we find the score for each job
- we substract each job score from the total score necessary
- any job with a ZSET score of 0 is a job that the candidate has all of the skills for

In [5]:
def index_job(conn, job_id, skills):
    pipeline = conn.pipeline(True)
    for skill in skills:
        pipeline.sadd(f'idx:skill:{skill}', job_id)
    pipeline.execute()

In [6]:
def find_jobs(conn, candidate_skills):
    skills = {}
    for skill in set(candidate_skills):
        skills['skill:' + skill] = 1
    
    job_scores = zunion(conn, skills)
    
    final_result = zintersect(conn, {job_scores: -1, 'jobs:req': 1})
    return conn.zrangebyscore(f'idx:{final_result}', 0, 0)